# Selecting a source from the catalog and creating a cutout image

The goal is to select a single source from the coadd measurements based on its properties and create a postage stamp image.

Based partly on:
* https://github.com/lsst-com/notebooks/blob/master/postage_stamp.ipynb

which was in turn guided by:
* https://pipelines.lsst.io/getting-started/index.html#getting-started-tutorials 
* https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Colour%20Images.ipynb

In [ ]:
import numpy as np
import random

import lsst.daf.persistence as dafPersist
from lsst.daf.persistence import Butler
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage

from astropy.visualization import ZScaleInterval

In [ ]:
# Set plotting defaults
%matplotlib inline
import matplotlib.pyplot as plt
zscale = ZScaleInterval()

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 20})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 3})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})

We'll look at data from the [HSC/SSP survey](http://hsc.mtk.nao.ac.jp/ssp/), which have been processed by the LSST stack. In particular, we'll focus on the ultra-deep ("UDEEP") coadds.

In [ ]:
##### If you want to edit the data set, tract, and patch yourself, change the numbers below appropriately
data_set = 'UDEEP'# HSC/SSP survey data include WIDE, DEEP, UDEEP fields
datadir = '/datasets/hsc/repo/rerun/DM-13666/' + data_set 
butler = Butler(datadir)

# We selected the "Ultra-deep (UDEEP)" data, and will choose a tract from the SXDS field (tract 8765):
tract = 8765 #8766
patch = '1,2' #'8,3'  # patch selected at random; you can select others if you like...
filt = 'HSC-G' # We'll select only a single filter for now.
dataid = {'filter': filt, 'tract':tract, 'patch':patch}

deep_coadd = butler.get('deepCoadd_forced_src', dataId=dataid)

### Read in the source catalog and the calibrated exposure

We'll use the `deepCoadd_forced_src` catalog, which contains forced photometry.

In [ ]:
# Source catalog:
my_src = butler.get('deepCoadd_forced_src', dataId=dataid)

# Calibrated exposure:
my_calexp = butler.get('deepCoadd_calexp', dataId=dataid)

# Extract the WCS and the calib object:
my_wcs = my_calexp.getWcs()
my_calib = my_calexp.getCalib()
my_calib.setThrowOnNegativeFlux(False) # For magnitudes

# Finally, get the reference table for the coadd sources:
my_src_ref = butler.get('deepCoadd_ref', dataId=dataid)

# To see the names of columns in this table, use:
# my_src_ref.schema.getNames()

The cell below takes advantage of slots defined in the schema.

The `getMagnitude` function below can return some `nan` and `inf` values. In the category of "best practices", we are suggesting that these _not_ be replaced with sentinel values, but rather that one filter the arrays when needed. (We achieve this with the `okmag` filter below.)

In [ ]:
# Data values can be accessed using explicit references (e.g., my_src['base_PsfFlux_flux'] for the PSF flux), 
# or the slot functionality that is built into the stack (e.g., my_src.getPsfFlux()). 

## This is just a demonstration of the slot functionality (uncomment to see result):
#np.testing.assert_equal(my_calib.getMagnitude(my_src['base_PsfFlux_flux']),
#                        my_calib.getMagnitude(my_src.getPsfFlux()))

# Extract the PSF flux, then calculate the PSF magnitude using the calib object:
psf_mag = my_calib.getMagnitude(my_src.getPsfInstFlux())

# Extract the CModel flux, then calculate the CModel magnitude using the calib object:
cm_mag = my_calib.getMagnitude(my_src.getModelInstFlux())

# Select only the objects that don't have NaN values in their mags:
okmag = (np.isfinite(psf_mag)) & (np.isfinite(cm_mag))

# Distribution of PSF magnitudes:
# If you have nan or inf values in your array, use the range argument to avoid searching for min and max
plt.figure()
plt.yscale('log', nonposy='clip')
plt.hist(psf_mag[okmag], bins=np.arange(17., 29., 0.25), range=(17., 29.))
#plt.hist(np.nan_to_num(psf_mag), bins=np.arange(15., 26., 0.25)) # Alternative
plt.xlabel('PSF Magnitude')
plt.ylabel('Counts')

# Difference between PSF and model mags, which is often used as a star/galaxy separation criterion. 
# Here we color-code by the stack's classification (0=Star, 1=Galaxy (extended source))
plt.figure()
plt.scatter(psf_mag[okmag], psf_mag[okmag] - cm_mag[okmag], c=my_src['base_ClassificationExtendedness_value'][okmag], cmap='coolwarm', s=6)
plt.colorbar().set_label('Classification')
plt.xlim(17., 29.)
plt.ylim(-1., 2.)
plt.xlabel('PSF Magnitude')
plt.ylabel('PSF - Model Magnitude')

Now select a source based on its properties; e.g., pick a bright star. We'll choose one that was used in creating the PSF, by using the `calib_psfUsed` flag.

In [ ]:
# Pick a bright star that was used to fit the PSF:
selection = my_src_ref['calib_psfUsed'] & np.isfinite(psf_mag) # Also require the PSF mag to not be NaN

# Pick one of the PSF stars at random:
sel_ind = np.where(selection)
ind0 = random.choice(np.arange(np.size(sel_ind)))
index = sel_ind[0][ind0]

print('Magnitude of selected star: ',psf_mag[index])

Extract the RA, Dec position of the star we've chosen, and create a `SpherePoint` object to be used to extract the postage stamp around this star:

In [ ]:
ra_target, dec_target = my_src['coord_ra'][index], my_src['coord_dec'][index] # Note: coordinates are in radians
radec = afwGeom.SpherePoint(ra_target, dec_target, afwGeom.radians) 

### Select a cutout image (using pixel coordinates for selection):

First, we will define a bounding box using methods in `afwGeom`, then we will extract a cutout using this bounding box via the `bbox` keyword.

In [ ]:
cutoutSize = afwGeom.ExtentI(300, 300) # size of cutout in pixels
xy = afwGeom.Point2I(my_wcs.skyToPixel(radec)) # central XY coordinate of our star's RA, Dec position

# Create the bounding box:
bbox = afwGeom.Box2I(xy - cutoutSize//2, cutoutSize)

# Full patch image
image = butler.get('deepCoadd_calexp', immediate=True, dataId=dataid)
# Because an entire tract shares a WCS, the corner of the patch (or cutout) isn't necessarily at (X,Y)=(0,0). Get the XY0 pixel values:
xy0 = image.getXY0() 

# Postage stamp image only, using the bbox defined above:
cutout_image = butler.get('deepCoadd_calexp_sub', bbox=bbox, immediate=True, dataId=dataid).getMaskedImage()
# Because an entire tract shares a WCS, the corner of the patch (or cutout) isn't necessarily at (X,Y)=(0,0). Get the XY0 pixel values:
xy0_cutout = cutout_image.getXY0() 

### Plot the image of the entire patch:

We'll highlight the PSF star we've selected with a magenta circle.

In [ ]:
vmin, vmax = zscale.get_limits(image.image.array)
# Get the dimensions of the image so we can set plot limits
imsize = image.getDimensions()
plt.imshow(image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
# Set the plot range to the dimensions:
plt.xlim(0,imsize[0])
plt.ylim(0,imsize[1])
plt.colorbar()
plt.xlabel('X (pix)')
plt.ylabel('Y (pix)')
plt.scatter(xy.getX()-xy0.getX(), xy.getY()-xy0.getY(), color='none', edgecolor='magenta', s=200, linewidth=3)

### Plot the image of the cutout:

In [ ]:
vmin, vmax = zscale.get_limits(cutout_image.image.array)
# Get the dimensions of the image so we can set plot limits
imsize = cutout_image.getDimensions()
plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
# Set the plot range to the dimensions:
plt.xlim(0,imsize[0])
plt.ylim(0,imsize[1])
plt.colorbar()
plt.xlabel('X (pix)')
plt.ylabel('Y (pix)')
plt.scatter(xy.getX()-xy0_cutout.getX(), xy.getY()-xy0_cutout.getY(), color='none', edgecolor='magenta', s=2000, linewidth=5)

### Now select nearby sources and overplot them on the image:

In [ ]:
# We'll use astropy utilities for finding nearby sources:
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
# Export the sources as an Astropy table:
src_tab = my_src.asAstropy()

# Create an Astropy "SkyCoord" object with the coordinates of all the sources:
src_coord = coord.SkyCoord(src_tab['coord_ra'],src_tab['coord_dec'])
# Select the PSF star plotted above:
selected_src_coord = src_coord[index]

In [ ]:
# Use the "separation" function for SkyCoord objects to select objects within a certain distance from the target star:

sep_arcmin = 0.5 # Desired separation in arcminutes
find_nearby = src_coord.separation(selected_src_coord) < sep_arcmin*u.arcmin
nearby_src = np.where(find_nearby & okmag) # added "okmag" to ensure only well-measured sources appear

print('Selected ',np.size(src_coord[nearby_src]),' sources')

In [ ]:
vmin, vmax = zscale.get_limits(cutout_image.image.array)
# Get the dimensions of the image so we can set plot limits
imsize = cutout_image.getDimensions()
plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
# Set the plot range to the dimensions:
plt.xlim(0,imsize[0])
plt.ylim(0,imsize[1])
plt.colorbar()
plt.xlabel('X (pix)')
plt.ylabel('Y (pix)')
plt.title(f'All detected sources within {sep_arcmin:5.2} arcmin of selected star')
plt.scatter(xy.getX()-xy0_cutout.getX(), xy.getY()-xy0_cutout.getY(), color='none', edgecolor='magenta', s=2000, linewidth=5)

for nearby in nearby_src[0]:
    ra_tmp, dec_tmp = my_src['coord_ra'][nearby], my_src['coord_dec'][nearby] # Radians
    radec_tmp = afwGeom.SpherePoint(ra_tmp, dec_tmp, afwGeom.radians)
    xy_tmp = afwGeom.Point2I(my_wcs.skyToPixel(radec_tmp))
    plt.scatter(xy_tmp.getX()-xy0_cutout.getX(), xy_tmp.getY()-xy0_cutout.getY(), color='none', edgecolor='cyan', s=400, linewidth=5)


### Separate point/extended sources

We'll use the `base_ClassificationExtendedness_value` -- this is set to 1 for things classified as extended, and 0 for point-like sources.

In [ ]:
gx = my_src['base_ClassificationExtendedness_value'] > 0.9 # extended sources have this flag set to 1
star = my_src['base_ClassificationExtendedness_value'] < 0.1 # for point sources, should be 0

In [ ]:
vmin, vmax = zscale.get_limits(cutout_image.image.array)
# Get the dimensions of the image so we can set plot limits
imsize = cutout_image.getDimensions()
plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
# Set the plot range to the dimensions:
plt.xlim(0,imsize[0])
plt.ylim(0,imsize[1])
plt.colorbar()
plt.xlabel('X (pix)')
plt.ylabel('Y (pix)')
plt.title(f'All detected sources within {sep_arcmin:5.2} arcmin of selected star')

nearby_star = np.where(find_nearby & okmag & star) # added "okmag" to ensure only well-measured sources appear
nearby_gx = np.where(find_nearby & okmag & gx) # added "okmag" to ensure only well-measured sources appear

for nearby in nearby_star[0]:
    ra_tmp, dec_tmp = my_src['coord_ra'][nearby], my_src['coord_dec'][nearby] # Radians
    radec_tmp = afwGeom.SpherePoint(ra_tmp, dec_tmp, afwGeom.radians)
    xy_tmp = afwGeom.Point2I(my_wcs.skyToPixel(radec_tmp))
    plt.scatter(xy_tmp.getX()-xy0_cutout.getX(), xy_tmp.getY()-xy0_cutout.getY(), color='none', edgecolor='cyan', s=400, linewidth=5)

plt.scatter(xy_tmp.getX()-xy0_cutout.getX(), xy_tmp.getY()-xy0_cutout.getY(), color='none', edgecolor='cyan', s=400, linewidth=5, label = 'star')

for nearby in nearby_gx[0]:
    ra_tmp, dec_tmp = my_src['coord_ra'][nearby], my_src['coord_dec'][nearby] # Radians
    radec_tmp = afwGeom.SpherePoint(ra_tmp, dec_tmp, afwGeom.radians)
    xy_tmp = afwGeom.Point2I(my_wcs.skyToPixel(radec_tmp))
    plt.scatter(xy_tmp.getX()-xy0_cutout.getX(), xy_tmp.getY()-xy0_cutout.getY(), color='none', edgecolor='red', s=400, linewidth=5)

plt.scatter(xy_tmp.getX()-xy0_cutout.getX(), xy_tmp.getY()-xy0_cutout.getY(), color='none', edgecolor='red', s=400, linewidth=5, label='galaxy')

plt.legend()